In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os

In [2]:
os.chdir('C:/Users/claud/Documents/GitHub/vivi_mazatlan')

In [3]:
#Función para hacer spatial join
def spatial_join(df1,df2):
    #Transformar coordenadas a geometría
    df1['geometry'] = df1.apply(lambda x: Point((float(x['lon']), float(x['lat']))), axis=1)
    df1=gpd.GeoDataFrame(df1, crs='EPSG:4326')
    #Hacer spatial join
    df1=gpd.sjoin(df1,df2,predicate='intersects',how='left')
    #Eliminar columnas
    df1=df1.drop(columns=['municipio','geometry','index_right'])
    #Transformar a dataframe
    df1=pd.DataFrame(df1)
    return df1

In [5]:
carto=gpd.read_file('data/25_sinaloa/conjunto_de_datos/25mun.shp')
carto.columns=carto.columns.str.lower()
carto=carto.rename(columns={'nomgeo':'nom_mun'})
#Filtrar Mazatlán
#Cambiar crs a 4326
carto=carto.to_crs('EPSG:4326')
#Filtrar Mazatlán
carto=carto[carto['nom_mun']=='Mazatlán']
carto

,cvegeo,cve_ent,cve_mun,nom_mun,geometry
11,25012,25,012,Mazatlán,"MULTIPOLYGON (((-106.41293 23.17304, -106.4130..."


In [6]:
#Leer datos de webscraping
#Viviendas
lamudi=pd.read_csv('output/viviendas_mazatlan_lamudi.csv')
easybroker=pd.read_csv('output/viviendas_mazatlan_easybroker.csv')
#Unir datos
viviendas=pd.concat([lamudi,easybroker],ignore_index=True)
viviendas

,oferta,recamaras,bathrooms,mts,lat,lon,fecha_consulta,fuente,precio,precio_m2,municipio
0,casa en venta en el cid,5.0,6.0,995.0,23.260946,-106.454759,2024-09-02 13:00:56.917298,lamudi,24500000.0,24623.115578,NaN
1,casa en fraccionamiento en venta en villa marina,4.0,3.0,270.0,23.290164,-106.466938,2024-09-02 13:00:56.917298,lamudi,7950000.0,29444.444444,NaN
2,casa en venta en balcones de loma linda,4.0,5.0,510.0,23.202841,-106.428832,2024-09-02 13:00:56.917298,lamudi,27980000.0,54862.745098,NaN
3,departamento en venta en el cid,3.0,3.0,163.0,23.259951,-106.446167,2024-09-02 13:00:56.917298,lamudi,6500000.0,39877.300613,NaN
4,casa en venta en real del valle,2.0,1.0,60.0,23.287662,-106.399028,2024-09-02 13:00:56.917298,lamudi,1350000.0,22500.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2908,condominios litoral ocean,NaN,NaN,NaN,23.206339,-106.428484,2024-09-02 12:27:21.140990,easybroker,4890000.0,NaN,mazatlan
2909,boca de mar una torre,NaN,NaN,NaN,23.232533,-106.434951,2024-09-02 12:27:21.140990,easybroker,6300000.0,NaN,mazatlan
2910,departamento en venta en torre eme.,2.0,2.0,NaN,23.218310,-106.422072,2024-09-02 12:27:21.140990,easybroker,8000000.0,NaN,mazatlan
2911,departamentos en marina campo de golf,2.0,2.0,86.0,23.281545,-106.456330,2024-09-02 12:27:21.140990,easybroker,4100000.0,47674.418605,mazatlan


In [7]:
viviendas=spatial_join(viviendas,carto)
#eliminar si nom_mun es nulo
viviendas=viviendas.dropna(subset=['nom_mun'])
viviendas

,oferta,recamaras,bathrooms,mts,lat,lon,fecha_consulta,fuente,precio,precio_m2,cvegeo,cve_ent,cve_mun,nom_mun
0,casa en venta en el cid,5.0,6.0,995.0,23.260946,-106.454759,2024-09-02 13:00:56.917298,lamudi,24500000.0,24623.115578,25012,25,012,Mazatlán
1,casa en fraccionamiento en venta en villa marina,4.0,3.0,270.0,23.290164,-106.466938,2024-09-02 13:00:56.917298,lamudi,7950000.0,29444.444444,25012,25,012,Mazatlán
2,casa en venta en balcones de loma linda,4.0,5.0,510.0,23.202841,-106.428832,2024-09-02 13:00:56.917298,lamudi,27980000.0,54862.745098,25012,25,012,Mazatlán
3,departamento en venta en el cid,3.0,3.0,163.0,23.259951,-106.446167,2024-09-02 13:00:56.917298,lamudi,6500000.0,39877.300613,25012,25,012,Mazatlán
4,casa en venta en real del valle,2.0,1.0,60.0,23.287662,-106.399028,2024-09-02 13:00:56.917298,lamudi,1350000.0,22500.000000,25012,25,012,Mazatlán
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2908,condominios litoral ocean,NaN,NaN,NaN,23.206339,-106.428484,2024-09-02 12:27:21.140990,easybroker,4890000.0,NaN,25012,25,012,Mazatlán
2909,boca de mar una torre,NaN,NaN,NaN,23.232533,-106.434951,2024-09-02 12:27:21.140990,easybroker,6300000.0,NaN,25012,25,012,Mazatlán
2910,departamento en venta en torre eme.,2.0,2.0,NaN,23.218310,-106.422072,2024-09-02 12:27:21.140990,easybroker,8000000.0,NaN,25012,25,012,Mazatlán
2911,departamentos en marina campo de golf,2.0,2.0,86.0,23.281545,-106.456330,2024-09-02 12:27:21.140990,easybroker,4100000.0,47674.418605,25012,25,012,Mazatlán


In [8]:
#Contar cvegeo unicos
viviendas['cvegeo'].nunique()

1

In [10]:
viviendas.info()
#Guardar tabla final
viviendas.to_csv('output/vivi_mazatlan_final.csv',index=False,encoding='latin1')

<class 'pandas.core.frame.DataFrame'>
Index: 2870 entries, 0 to 2912
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   oferta          2870 non-null   object 
 1   recamaras       2652 non-null   float64
 2   bathrooms       2529 non-null   float64
 3   mts             2632 non-null   float64
 4   lat             2870 non-null   float64
 5   lon             2870 non-null   float64
 6   fecha_consulta  2870 non-null   object 
 7   fuente          2870 non-null   object 
 8   precio          2870 non-null   float64
 9   precio_m2       2632 non-null   float64
 10  cvegeo          2870 non-null   object 
 11  cve_ent         2870 non-null   object 
 12  cve_mun         2870 non-null   object 
 13  nom_mun         2870 non-null   object 
dtypes: float64(7), object(7)
memory usage: 336.3+ KB
